In [ ]:
import yfinance as yf
import openpyxl as xl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
CSV_FILE = './csv/your_file.csv'
TITLE_SECTOR_DISTRIBUTIONS = 'Account Distributions (December 2022)'
TITLE_ASSET_CLASSES = 'Asset Class Distribution (December 2022)'

In [ ]:
df = pd.read_csv(CSV_FILE, header=2)

In [ ]:
sector_weighting_conversions = {
    'realestate': 'Real Estate',
    'utilities': 'Utilities',
    'consumer_cyclical': 'Consumer Cyclical',
    'consumer_defensive': 'Consumer Defensive',
    'energy': 'Energy',
    'financial_services': 'Financial Services',
    'healthcare': 'Healthcare',
    'industrials': 'Industrials',
    'technology': 'Technology',
    'basic_materials': 'Basic Materials',
    'communication_services': 'Communication Services'
                                }

ticker_list = []
sector_weightings_in_account = {}
asset_classes_in_account = {}


def add_to_asset_classes(asset_class, amount):
    if type(amount) == str:
        amount = amount.strip('%')
    if asset_class not in asset_classes_in_account.keys():
        asset_classes_in_account[asset_class] = float(amount)
    elif asset_class in asset_classes_in_account.keys():
        asset_classes_in_account[asset_class] += float(amount)


def add_to_sector_weightings(sector, amount):
    if type(amount) == str:
        amount = amount.strip('%')
    if sector not in sector_weightings_in_account.keys():
        sector_weightings_in_account[sector] = float(amount)
    elif sector in sector_weightings_in_account.keys():
        sector_weightings_in_account[sector] += float(amount)


def add_to_sector_weightings_fund(weights_dict, total_amount_of_acct):
    if type(total_amount_of_acct) == str:
        total_amount_of_acct = total_amount_of_acct.strip('%')
    new_weightings_dict = {}
    check_total = float(0)
    for key in weights_dict.keys():
        holding_value = weights_dict[key]
        new_weightings_dict[key] = float(total_amount_of_acct) * holding_value
    for sector in new_weightings_dict.keys():
        check_total += new_weightings_dict[sector]
        if sector not in sector_weightings_in_account.keys():
            sector_weightings_in_account[sector] = new_weightings_dict[sector]
        elif sector in sector_weightings_in_account.keys():
            sector_weightings_in_account[sector] += new_weightings_dict[sector]
    print(f'Checking totals: Passed total was {total_amount_of_acct}, parsed was {check_total}.')
    remainder = float(total_amount_of_acct) - check_total
    print(f'Passing {remainder} to "Fund Remainder."')
    add_to_sector_weightings('Fund Remainder', remainder)


for row in df.index:
    ticker = df.at[row, 'Symbol']
    ticker_list.append(ticker)
    try:
        security_type = df.at[row, 'Security Type']
        if security_type == 'Equity':
            ticker_lookup = yf.Ticker(ticker)
            quote_type = ticker_lookup.info['quoteType']
            if quote_type == 'EQUITY':
                sector = ticker_lookup.info['sector']
                df.at[row, 'Sector'] = sector
                add_to_sector_weightings(sector, df.at[row, '% Of Account'])
                add_to_asset_classes(security_type, df.at[row, '% Of Account'])
                print(f'Added equity {ticker}.')
            elif quote_type == 'ETF':
                df.at[row, 'Security Type'] = 'Preferred Stock'
                df.at[row, 'Sector'] = 'Preferred Stock'
                add_to_sector_weightings('Fixed Income', df.at[row, '% Of Account'])
                add_to_asset_classes('Fixed Income', df.at[row, '% Of Account'])
                print(f'Adding preferred stock {ticker}.')
        elif security_type == 'ETFs & Closed End Funds' or security_type == 'Mutual Fund':
            ticker_lookup = yf.Ticker(ticker)
            weightings_dict = {}
            for item in ticker_lookup.info['sectorWeightings']:
                 for key, value in item.items():
                    new_key = sector_weighting_conversions[key]
                    weightings_dict[new_key] = value
            df.at[row, 'Sector'] = weightings_dict
            total_of_acct = df.at[row, '% Of Account']
            add_to_sector_weightings_fund(weightings_dict, total_of_acct)
            add_to_asset_classes(security_type, df.at[row, '% Of Account'])
            print(f'Adding ETF/Mutual Fund {ticker}.')
        elif security_type == 'Option':
            add_to_sector_weightings('Options', df.at[row, '% Of Account'])
            add_to_asset_classes(security_type, df.at[row, '% Of Account'])
            print(f'Adding option {ticker}')
        elif security_type == 'Fixed Income':
            add_to_sector_weightings('Fixed Income', df.at[row, '% Of Account'])
            add_to_asset_classes(security_type, df.at[row, '% Of Account'])
            print(f'Adding fixed income {ticker}')
        elif security_type == 'Cash and Money Market':
            cash_amount = df.at[row, '% Of Account']
            cash_amount = cash_amount.strip('%')
            sector_weightings_in_account['Cash'] = float(cash_amount)
            add_to_asset_classes(security_type, df.at[row, '% Of Account'])
        else:
            print(f'Skipping {ticker}')
    except Exception as e:
            print(f'Skipping {ticker} - Error: {e}')

print(sector_weightings_in_account)

In [ ]:
total = float(0)
for key in sector_weightings_in_account.keys():
    total += sector_weightings_in_account[key]
print(total)

In [ ]:
sector_weightings = pd.DataFrame(sector_weightings_in_account, columns=sector_weightings_in_account.keys(), index=['Account %'])
sector_weightings = sector_weightings.transpose()
sector_weightings = sector_weightings.sort_index()
sector_weightings

In [ ]:
plt.pie(x=sector_weightings['Account %'], labels=sector_weightings.index, autopct='%.2f', radius=1.8, shadow=True)
plt.title(TITLE_SECTOR_DISTRIBUTIONS, pad=100)
plt.show()

In [ ]:
asset_classes = pd.DataFrame(asset_classes_in_account, columns=asset_classes_in_account.keys(), index=['Account %'])
asset_classes = asset_classes.transpose()
asset_classes = asset_classes.sort_index()
asset_classes

In [ ]:
plt.pie(x=asset_classes['Account %'], labels=asset_classes.index, autopct='%.2f', radius=1.8, shadow=True)
plt.title(TITLE_ASSET_CLASSES, pad=100)
plt.show()